In [61]:
import pandas as pd

In [62]:
glogs = pd.read_csv(r'C:\Users\travi\OneDrive\Documents\PennState\classes\2025 spring\DS 440\game_logs.csv')

# Convert date column to datetime, filters out dates before 2005 
glogs['date'] = pd.to_datetime(glogs['date'], format=r'%Y%m%d')
glogs = glogs[glogs['date'].dt.year >= 2005]

# Specify columns to keep for analysis
glogs = glogs[['date', 'day_of_week', 'v_name', 'v_league', 'v_game_number', 'h_name', 'h_league', 
               'h_game_number', 'v_score', 'h_score', 'day_night', 'park_id', 'attendance']]

# Remove international ballparks that were only used for one random game
glogs = glogs[~glogs['park_id'].isin(['SYD01', 'TOK01', "SJU01", 'FTB01', 'LBV01'])]
glogs.head()

C:\Users\travi\AppData\Local\Temp\ipykernel_3432\2179221868.py:1: DtypeWarning: Columns (13,14,15,16,20,21,82,84,86,88,94,95,96,97,98,99,100,101,106,107,109,110,112,113,115,116,118,119,121,122,124,125,127,128,130,131,133,134,136,137,139,140,142,143,145,146,148,149,151,152,154,155,157,158,161) have mixed types. Specify dtype option on import or set low_memory=False.
  glogs = pd.read_csv(r'C:\Users\travi\OneDrive\Documents\PennState\classes\2025 spring\DS 440\game_logs.csv')


,date,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,day_night,park_id,attendance
142751,2005-04-03,Sun,BOS,AL,1,NYA,AL,1,2,9,N,NYC16,54818.0
142752,2005-04-04,Mon,OAK,AL,1,BAL,AL,1,0,4,D,BAL12,48271.0
142753,2005-04-04,Mon,CLE,AL,1,CHA,AL,1,0,1,D,CHI12,38141.0
142754,2005-04-04,Mon,KCA,AL,1,DET,AL,1,2,11,D,DET05,44105.0
142755,2005-04-04,Mon,MIN,AL,1,SEA,AL,1,1,5,D,SEA03,46249.0


In [63]:
# Rename stadium names in park_id column
# STL09 = Busch Stadium II
# STL10 = Busch Stadium III
# NYC16 = Old Yankee Stadium
# NYC17 = Shea Stadium
# NYC20 = Citi Field
# NYC21 = New Yankee Stadium
# WAS10 = RFK Stadium
# WAS11 = Nationals Park
# MIA01 = Sun Life Stadium
# MIA02 = Marlins Park
# MIN03 = Metrodome
# MIN04 = Target Field
glogs['park_id'] = glogs['park_id'].replace({'ANA01': 'Angel Stadium',
                                             'ARL02': 'Globe Life Field',
                                             'ATL02': 'Turner Field',
                                             'BAL12': 'Camden Yards',
                                             'BOS07': 'Fenway Park',
                                             'CHI11': 'Wrigley Field',
                                             'CHI12': 'US Cellular Field',
                                             'CIN09': 'Great American Ballpark',
                                             'CLE08': 'Progressive Field',
                                             'DEN02': 'Coors Field',
                                             'DET05': 'Comerica Park',
                                             'HOU03': 'Minute Maid Park',
                                             'KAN06': 'Koffman Stadium',
                                             'LOS03': 'Dodger Stadium',
                                             'MIA01': 'Sun Life Stadium',
                                             'MIA02': 'Marlins Park',
                                             'MIL06': 'Miller Park',
                                             'MIN03': 'Metrodome',
                                             'MIN04': 'Target Field',
                                             'NYC16': 'Old Yankee Stadium',
                                             'NYC17': 'Shea Stadium',
                                             'NYC20': 'Citi Field',
                                             'NYC21': 'New Yankee Stadium',
                                             'OAK01': 'Coliseum',
                                             'PHI13': 'Citizens Bank Park',
                                             'PHO01': 'Chase Field',
                                             'PIT08': 'PNC Park',
                                             'SAN02': 'Petco Park',
                                             'SEA03': 'Safeco Field',
                                             'SFO03': 'AT&T Park',
                                             'STL09': 'Busch Stadium II',
                                             'STL10': 'Busch Stadium III',
                                             'STP01': 'Tropicana Field',
                                             'TOR02': 'Rogers Centre',
                                             'WAS10': 'RFK Stadium',
                                             'WAS11': 'Nationals Park',
                                             })
glogs.head()

,date,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,day_night,park_id,attendance
142751,2005-04-03,Sun,BOS,AL,1,NYA,AL,1,2,9,N,Old Yankee Stadium,54818.0
142752,2005-04-04,Mon,OAK,AL,1,BAL,AL,1,0,4,D,Camden Yards,48271.0
142753,2005-04-04,Mon,CLE,AL,1,CHA,AL,1,0,1,D,US Cellular Field,38141.0
142754,2005-04-04,Mon,KCA,AL,1,DET,AL,1,2,11,D,Comerica Park,44105.0
142755,2005-04-04,Mon,MIN,AL,1,SEA,AL,1,1,5,D,Safeco Field,46249.0


In [64]:
# Rename team abbreviations for consistency with conventional abbreviations
glogs['v_name'] = glogs['v_name'].replace({'ANA':'LAA',
                         'CHA':'CHW',
                         'CHN':'CHC',
                         'FLO':'MIA',
                         'KCA':'KC',
                         'LAN':'LAD',
                         'NYA':'NYY',
                         'NYN':'NYM',
                         'SDN':'SD',
                         'SFN':'SF',
                         'SLN':'STL',
                         'TBA':'TB',
                         'WAS':'WSH'})
glogs['h_name'] = glogs['h_name'].replace({'ANA':'LAA',
                         'CHA':'CHW',
                         'CHN':'CHC',
                         'FLO':'MIA',
                         'KCA':'KC',
                         'LAN':'LAD',
                         'NYA':'NYY',
                         'NYN':'NYM',
                         'SDN':'SD',
                         'SFN':'SF',
                         'SLN':'STL',
                         'TBA':'TB',
                         'WAS':'WSH'})
glogs['h_name'].sort_values().unique()

array(['ARI', 'ATL', 'BAL', 'BOS', 'CHC', 'CHW', 'CIN', 'CLE', 'COL',
       'DET', 'HOU', 'KC', 'LAA', 'LAD', 'MIA', 'MIL', 'MIN', 'NYM',
       'NYY', 'OAK', 'PHI', 'PIT', 'SD', 'SEA', 'SF', 'STL', 'TB', 'TEX',
       'TOR', 'WSH'], dtype=object)

In [65]:
# Add column 'city'
glogs['location'] = glogs['h_name'].map({'ARI': 'Arizona', 'ATL': 'Atlanta', 'BAL': 'Baltimore', 'BOS': 'Boston', 'CHC': 'Chicago',
                                         'CHW': 'Chicago', 'CIN': 'Cincinatti', 'CLE': 'Cleveland', 'COL': 'Colorado', 'DET': 'Detroit',
                                         'HOU': 'Houston', 'KC': 'Kansas City', 'LAA': 'Los Angeles', 'LAD': 'Los Angeles', 'MIA': 'Miami',
                                         'MIL': 'Miluakee', 'MIN': 'Minnesota', 'NYM': 'New York', 'NYY': 'New York', 'OAK': 'Oakland',
                                         'PHI': 'Philadelphia', 'PIT': 'Pittsburgh', 'SD': 'San Diego', 'SEA': 'Seattle',
                                         'SF': 'San Francisco', 'STL': 'St. Louis', 'TB': 'Tampa Bay', 'TEX': 'Texas', 'TOR': 'Toronto',
                                         'WSH': 'Washington, D.C.'
})
glogs

,date,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,day_night,park_id,attendance,location
142751,2005-04-03,Sun,BOS,AL,1,NYY,AL,1,2,9,N,Old Yankee Stadium,54818.0,New York
142752,2005-04-04,Mon,OAK,AL,1,BAL,AL,1,0,4,D,Camden Yards,48271.0,Baltimore
142753,2005-04-04,Mon,CLE,AL,1,CHW,AL,1,0,1,D,US Cellular Field,38141.0,Chicago
142754,2005-04-04,Mon,KC,AL,1,DET,AL,1,2,11,D,Comerica Park,44105.0,Detroit
142755,2005-04-04,Mon,MIN,AL,1,SEA,AL,1,1,5,D,Safeco Field,46249.0,Seattle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171902,2016-10-02,Sun,MIL,NL,162,COL,NL,162,6,4,D,Coors Field,27762.0,Colorado
171903,2016-10-02,Sun,NYM,NL,162,PHI,NL,162,2,5,D,Citizens Bank Park,36935.0,Philadelphia
171904,2016-10-02,Sun,LAD,NL,162,SF,NL,162,1,7,D,AT&T Park,41445.0,San Francisco
171905,2016-10-02,Sun,PIT,NL,162,STL,NL,162,4,10,D,Busch Stadium III,44615.0,St. Louis


In [66]:
# Add time column - not needed anymore
# glogs['time'] = glogs['day_night'].apply(lambda x: pd.to_datetime('13:00:00') if x == 'D' else pd.to_datetime('19:00:00'))
# glogs['time'] = glogs['time'].dt.time
# glogs.head()

In [ ]:
# Read in weather data
weather_PHI = pd.read_csv(r'C:\Users\travi\OneDrive\Documents\PennState\classes\2025 spring\DS 440\GitHubRepo\DS440_project\weather_PHI.csv')
weather_PHI['date'] = pd.to_datetime(weather_PHI['date'])
weather_PHI['location'] = 'Philadelphia'

weather_NYM = pd.read_csv(r'C:\Users\travi\OneDrive\Documents\PennState\classes\2025 spring\DS 440\GitHubRepo\DS440_project\weather_NYM.csv')
weather_NYM['date'] = pd.to_datetime(weather_NYM['date'])
weather_NYM['location'] = 'New York'

weather_ATL = pd.read_csv(r'C:\Users\travi\OneDrive\Documents\PennState\classes\2025 spring\DS 440\GitHubRepo\DS440_project\weather_ATL.csv')
weather_ATL['date'] = pd.to_datetime(weather_ATL['date'])
weather_ATL['location'] = 'Atlanta'

weather_MIA = pd.read_csv(r'C:\Users\travi\OneDrive\Documents\PennState\classes\2025 spring\DS 440\GitHubRepo\DS440_project\weather_MIA.csv')
weather_MIA['date'] = pd.to_datetime(weather_MIA['date'])
weather_MIA['location'] = 'Miami'

weather_WSH = pd.read_csv(r'C:\Users\travi\OneDrive\Documents\PennState\classes\2025 spring\DS 440\GitHubRepo\DS440_project\weather_WSH.csv')
weather_WSH['date'] = pd.to_datetime(weather_WSH['date'])
weather_WSH['location'] = 'Washington, D.C.'


# Concatenate NL East weather vertically
weather_nl_east = pd.concat([weather_PHI, weather_NYM, weather_ATL, weather_MIA, weather_WSH], ignore_index=True)
weather_nl_east

,date,temperature_2m_max (°F),rain_sum (inch),location
0,2005-04-03,49.6,0.272,Philadelphia
1,2005-04-04,59.8,0.000,Philadelphia
2,2005-04-05,66.0,0.000,Philadelphia
3,2005-04-06,76.8,0.000,Philadelphia
4,2005-04-07,73.7,0.122,Philadelphia
...,...,...,...,...
21005,2016-09-29,70.7,1.461,"Washington, D.C."
21006,2016-09-30,66.9,0.094,"Washington, D.C."
21007,2016-10-01,68.0,0.181,"Washington, D.C."
21008,2016-10-02,71.7,0.000,"Washington, D.C."


In [88]:
# Merge NL East weather data with game logs
merged = pd.merge(glogs, weather_nl_east, on=['date', 'location'], how='left')
merged

,date,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,day_night,park_id,attendance,location,temperature_2m_max (°F),rain_sum (inch)
0,2005-04-03,Sun,BOS,AL,1,NYY,AL,1,2,9,N,Old Yankee Stadium,54818.0,New York,47.6,0.063
1,2005-04-04,Mon,OAK,AL,1,BAL,AL,1,0,4,D,Camden Yards,48271.0,Baltimore,NaN,NaN
2,2005-04-04,Mon,CLE,AL,1,CHW,AL,1,0,1,D,US Cellular Field,38141.0,Chicago,NaN,NaN
3,2005-04-04,Mon,KC,AL,1,DET,AL,1,2,11,D,Comerica Park,44105.0,Detroit,NaN,NaN
4,2005-04-04,Mon,MIN,AL,1,SEA,AL,1,1,5,D,Safeco Field,46249.0,Seattle,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29135,2016-10-02,Sun,MIL,NL,162,COL,NL,162,6,4,D,Coors Field,27762.0,Colorado,NaN,NaN
29136,2016-10-02,Sun,NYM,NL,162,PHI,NL,162,2,5,D,Citizens Bank Park,36935.0,Philadelphia,70.5,0.039
29137,2016-10-02,Sun,LAD,NL,162,SF,NL,162,1,7,D,AT&T Park,41445.0,San Francisco,NaN,NaN
29138,2016-10-02,Sun,PIT,NL,162,STL,NL,162,4,10,D,Busch Stadium III,44615.0,St. Louis,NaN,NaN


In [ ]:
glogs

142751    Old Yankee Stadium
142752          Camden Yards
142753     US Cellular Field
142754         Comerica Park
142755          Safeco Field
                 ...        
171902           Coors Field
171903    Citizens Bank Park
171904             AT&T Park
171905     Busch Stadium III
171906        Nationals Park
Name: park_id, Length: 29140, dtype: object